In [ ]:
import os
import json

from pymongo import MongoClient

# Prep

Set environment variables and setup simulated vasp run

In [ ]:
#set the path to vasp pseudopotential files
#os.environ["VASP_PSP_DIR"] = os.path.join(os.getcwd(), "reference_files")

# Structure and input set

In [ ]:
from pymatgen.util.testing import PymatgenTest
from pymatgen.io.vasp.sets import MPRelaxSet

struct_si = PymatgenTest.get_structure("Si")
vis = MPRelaxSet(struct_si, force_gamma=True)

# Define Fireworks and Workflow

In [ ]:
from fireworks import Workflow
from matmethods.vasp.fireworks.core import OptimizeFW, StaticFW, TransmuterFW


vasp_cmd=None
db_file=">>db_file<<"

fws = []
fw1 = OptimizeFW(struct_si, vasp_input_set=vis, vasp_cmd=vasp_cmd, db_file=db_file)
fws.append(fw1)

In [ ]:
from pymatgen.analysis.elasticity.strain import Deformation
from fireworks import Firework
from matmethods.vasp.workflows.base.elastic import PassStressStrainData

# Normal deformation
normal_deformation = Deformation.from_index_amount((0,0), 0.01)

#Shear
#shear_deformation = Deformation.from_index_amount((0,1), 0.01)

deformation_vasp_params = {"user_incar_settings": {"ISIF": 2, "IBRION": 2, "NSW": 99, "LAECHG": False,
                                           "LHVAR": False, "ALGO": "Fast", "LWAVE": False}}
    
# deformation firework
fw = TransmuterFW(name="normal deformation",
                  structure=struct_si,
                  transformations=['DeformStructureTransformation'],
                  transformation_params=[{"deformation": normal_deformation.tolist()}],
                  copy_vasp_outputs=True,
                  db_file=db_file,
                  vasp_cmd=vasp_cmd,
                  parents=fws[0],
                  vasp_input_params=deformation_vasp_params
                 )

# add task to pass the stress strain data
fw.spec['_tasks'].append(PassStressStrainData(deformation=normal_deformation.tolist()).to_dict())
fws.append(fw)

In [ ]:
from matmethods.vasp.workflows.base.elastic import AnalyzeStressStrainData

# stress-strain analyzer firework
fws.append(Firework(AnalyzeStressStrainData(structure=struct_si, db_file=db_file),
                    name="Analyze Elastic Data", parents=fws[1:],
                    spec={"_allow_fizzled_parents": True}))

wfname = "{}:{}".format(struct_si.composition.reduced_formula, "elastic constants")

wf = Workflow(fws, name=wfname)

In [ ]:
len(wf.as_dict()["fws"])

In [ ]:
from matmethods.vasp.vasp_powerups import use_fake_vasp

#reference_dir = os.path.join(os.getcwd(), "reference_files")
reference_dir = os.path.abspath(os.path.join("..","..","matproj/MatMethods/matmethods/vasp/tests", 
                                             "reference_files"))

si_ref_dirs = {"structure optimization": os.path.join(reference_dir, "Si_structure_optimization"), 
               "normal deformation": os.path.join(reference_dir, "Si_normal_deformation")}

wf_fake = use_fake_vasp(wf, si_ref_dirs)

# Connect to Launchpad and reset

In [ ]:
import datetime
from fireworks import LaunchPad

lp = LaunchPad.from_file("my_launchpad.yaml")
lp.reset(datetime.datetime.now().strftime('%Y-%m-%d'))

# Submit Workflow

In [ ]:
lp.add_wf(wf_fake)

In [ ]:
lp.get_fw_ids()

# Run

In [ ]:
from fireworks import FWorker
from fireworks.core.rocket_launcher import rapidfire, launch_rocket

db_file = os.path.join(os.getcwd(), "db.json")
#rapidfire(lp, fworker=FWorker(env={"db_file": db_file}))
launch_rocket(lp, fworker=FWorker(env={"db_file": db_file}))

In [ ]:
for id in lp.get_fw_ids():
    fw = lp.get_fw_by_id(id)
    print fw.name, fw.fw_name, fw.state

# Retrieve and Verify

In [ ]:
def get_task_collection(db_file):
    """
    connect to the database and return task collection
    """
    with open(db_file) as f:
        creds = json.loads(f.read())
        conn = MongoClient(creds["host"], creds["port"])
        db = conn[creds["database"]]
        if "admin_user" in creds:
            db.authenticate(creds["admin_user"], creds["admin_password"])
        return db[creds["collection"]]

In [ ]:
from pymongo import DESCENDING

d1 = get_task_collection(db_file).find_one({"task_label": "structure optimization"}, sort=[("_id", DESCENDING)])

d2 = get_task_collection(db_file).find_one({"task_label": "normal deformation"}, sort=[("_id", DESCENDING)])

In [ ]:
d1["output"]["bandgap"] # 0.65

# Debug

In [ ]:
!rm -rf launcher*